Autor: **Patryk Wittbrodt**  
**WZ_INIS6 50933** 

# Zadanie 4

*Napisz co to są **wątki** i do czego się je używa. Wymyśl dwie aplikacje działające **wielowątkowo** (samo wypisywanie informacji na ekranie to za mało) i je zaimplementuj w **języku Python**.*

### Wątki

> _**Wątek** to osobna jednostka wykonawcza działająca w obrębie tego samego procesu._

> _**Proces** to taki worek na wątki. W jego obrębie może działać wiele wątków (ale nie mniej niż jeden), które współdzielą pamięć._

<blockquote><i>Praca z wątkami jest zazwyczaj realizowana na trzy sposoby:
<ul>
    <li>poprzez użycie klasy <b>Thread</b>,</li>
    <li>poprzez dziedziczenie z klasy <b>Thread</b>,</li>
    <li>poprzez pulę wątków.</li>
</ul></i>
</blockquote>

*(źródło: [mmazurek.dev](https://mmazurek.dev/watki-i-procesy-w-pythonie/))*

#### Przykład nr 1: Tworzenie plików .txt i podmiana ich treści

Jako przykład posłuży nam 10 plików txt z błędem, który będziemy chcieli poprawić. Treść plików przed zmianą:

> nzwa: lorem ipsum

Chcemy zmienić "nzwa" na "nazwa":

> nazwa: lorem ipsum

In [2]:
# importowanie bibliotek
from threading import Thread
from time import perf_counter

# ścieżki do plików, których treść będzie podmieniana
filenames = [
    'txt/test1.txt',
    'txt/test2.txt',
    'txt/test3.txt',
    'txt/test4.txt',
    'txt/test5.txt',
    'txt/test6.txt',
    'txt/test7.txt',
    'txt/test8.txt',
    'txt/test9.txt',
    'txt/test10.txt',
]

# stworzenie funkcji, która stworzy nam pliki .txt z błędem w pisowni (bez użycia wątków, zobaczymy różnicę w czasie wykonania)
def createTxt():
    start_time = perf_counter()

    for filename in filenames:
        with open(filename, 'w') as f:
            f.write('nzwa: lorem ipsum')
    
    end_time = perf_counter()

    print(f'Tworzenie plików .txt bez użycia wątków trwało {end_time- start_time :0.2f}s.')

# stworzenie funkcji, która będzie podmieniać treść pliku
def replace(filename, substr, new_substr):
    print(f'Obsługuję plik {filename}')
    
    # pobieranie treści pliku
    with open(filename, 'r') as f:
        content = f.read()

    # podmiana wspomnianej frazy na nową
    content = content.replace(substr, new_substr)

    # zapisanie zmian
    with open(filename, 'w') as f:
        f.write(content)

Tworzenie plików .txt:

In [6]:
# tworzenie plików .txt
createTxt()

Tworzenie plików .txt bez użycia wątków trwało 0.02s.


Wynik:

<img src="https://i.imgur.com/WmbmRGC.png">

In [14]:
# podmiana treści z użyciem wątków

def main():

    # tworzenie wątków
    threads = [Thread(target=replace, args=(filename, 'nzwa', 'nazwa'))
            for filename in filenames]

    # rozpoczęcie wątków
    for thread in threads:
        thread.start()

    # czekanie, aż wątki zakończą swoje zadanie
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    start_time = perf_counter()

    main()

    end_time = perf_counter()
    print(f'Wykonanie programu z użyciem wątków trwało {end_time- start_time :0.2f}s.')

Obsługuję plik txt/test1.txt
Obsługuję plik txt/test2.txt
Obsługuję plik txt/test3.txt
Obsługuję plik txt/test4.txt
Obsługuję plik txt/test5.txt
Obsługuję plik txt/test6.txt
Obsługuję plik txt/test7.txt
Obsługuję plik txt/test8.txt
Obsługuję plik txt/test9.txt
Obsługuję plik txt/test10.txt
Wykonanie programu z użyciem wątków trwało 0.01s.


Wynik:  

<img src="https://i.imgur.com/bTgk3r6.png">

#### Przykład nr 2: Pingowanie urządzeń w sieci lokalnej

Pingowanie w zwykłej pętli bez użycia wątków byłoby bardzo niewydajne - musielibyśmy czekać na każdy ping po kolei

In [44]:
# importowanie bibliotek, Thread już jest zaimportowany w poprzednim kodzie
import os

# tworzenie klasy z wątkiem
class ip_check(Thread):
   def __init__ (self, ip):
      Thread.__init__(self)
      self.ip = ip
      self.__successful_pings = -1 # domyślna wartość w razie błędów

   def run(self):
      ping_out = os.popen("ping " + self.ip, 'r').read() # wykonujemy ping i czytamy wynik (na Linuxie polecenie ping daje więcej opcji i kod mógłby się wykonać jeszcze szybciej)

      self.__successful_pings = 1 # domyślnie uznajemy, że ping zakończył się sukcesem

      # jeśli jednak host nie został znaleziony, przypiszemy mu taką wartość
      if "unreachable" in ping_out:
         self.__successful_pings = 0

   # pobieranie statusu
   def status(self):
      if self.__successful_pings == 0:
         return "brak odpowiedzi"
      elif self.__successful_pings == 1:
         return "aktywny"
      else:
         return "błąd"

check_results = []

# pingujemy od 192.168.0.1 do 192.168.0.15
for suffix in range(1,16):
   ip = "192.168.0." + str(suffix)
   current = ip_check(ip)
   check_results.append(current)
   current.start()

# wyświetlamy wynik
for el in check_results:
   el.join()
   print("Status adresu", el.ip, ":", el.status())

Status adresu 192.168.0.1 : aktywny
Status adresu 192.168.0.2 : aktywny
Status adresu 192.168.0.3 : brak odpowiedzi
Status adresu 192.168.0.4 : brak odpowiedzi
Status adresu 192.168.0.5 : brak odpowiedzi
Status adresu 192.168.0.6 : brak odpowiedzi
Status adresu 192.168.0.7 : brak odpowiedzi
Status adresu 192.168.0.8 : brak odpowiedzi
Status adresu 192.168.0.9 : brak odpowiedzi
Status adresu 192.168.0.10 : brak odpowiedzi
Status adresu 192.168.0.11 : brak odpowiedzi
Status adresu 192.168.0.12 : brak odpowiedzi
Status adresu 192.168.0.13 : brak odpowiedzi
Status adresu 192.168.0.14 : aktywny
Status adresu 192.168.0.15 : brak odpowiedzi


### Bibliografia

- https://mmazurek.dev/watki-i-procesy-w-pythonie/